In [1]:
import random
import numpy as np
import tensorflow as tf
import mlflow.keras
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, PredefinedSplit
from sklearn.metrics import accuracy_score

from ml_investing_wne import config
from ml_investing_wne.utils import get_logger
from ml_investing_wne.experiment_factory import create_asset, experiment_factory

random.seed(config.seed)
np.random.seed(config.seed)
tf.random.set_seed(config.seed)

build_model for keras_tuner_CNN_LSTM not implemented!


In [2]:
print(config.currency)
print(config.freq) 
asset = create_asset()
experiment= experiment_factory(asset).get_experiment()

ETHUSDT
720min


In [3]:
print(config.fixed_barrier)
print(config.RUN_SUBTYPE)

0.025
dollar_bars_triple_barrier


In [4]:
# change all column types to numeric
experiment.df = experiment.df.apply(pd.to_numeric, errors='ignore')
experiment.df.dtypes

open             float64
close            float64
high             float64
low              float64
volume           float64
y_pred           float64
EMA_5            float64
VAR_5            float64
EMA_10           float64
VAR_10           float64
EMA_15           float64
VAR_15           float64
EMA_20           float64
VAR_20           float64
EMA_50           float64
VAR_50           float64
MACD_12_26_9     float64
MACDh_12_26_9    float64
MACDs_12_26_9    float64
RSI_14           float64
RSI_10           float64
RSI_6            float64
STOCHk_14_3_3    float64
STOCHd_14_3_3    float64
WILLR_14         float64
BBL_5_2.0        float64
BBM_5_2.0        float64
BBU_5_2.0        float64
BBB_5_2.0        float64
BBP_5_2.0        float64
roc_1            float64
CMF_20           float64
MFI_14           float64
hour_sin         float64
hour_cos         float64
weekday_sin      float64
weekday_cos      float64
dtype: object

In [5]:
experiment.df.head(2)

,open,close,high,low,volume,y_pred,EMA_5,VAR_5,EMA_10,VAR_10,...,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,roc_1,CMF_20,MFI_14,hour_sin,hour_cos,weekday_sin,weekday_cos
datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-18 05:51:52.380,1048.03,962.01,1054.0,924.21,50953.71749,1.0,979.390930,2783.67603,975.971791,7214.674823,...,1089.986948,18.959498,0.322019,-0.082060,-0.053826,43.198266,0.979084,0.203456,1.224647e-16,-1.0
2018-01-18 11:26:38.123,962.01,1032.80,1033.0,930.86,51084.52676,1.0,997.193953,1651.64463,986.304193,8026.897649,...,1088.265813,14.317103,0.618529,0.073586,-0.038363,43.260509,0.136167,-0.990686,1.224647e-16,-1.0


In [6]:
# experiment.df.drop(columns=['to_keep'],inplace=True)

In [7]:
train = experiment.df[experiment.df.index < config.train_end]
val = experiment.df[(experiment.df.index >= config.train_end) & (experiment.df.index < config.val_end)]
test = experiment.df[experiment.df.index >= config.val_end]
train_val = experiment.df[experiment.df.index < config.val_end]
train_test = pd.concat([train, test])

In [8]:
train_indices = np.where(train.index < config.train_end, -1, 0)
val_indices = np.where(val.index < config.val_end, 0, 1)
test_indices = np.where(test.index >= config.val_end, 0, 1)

In [9]:
split_indices = np.append(train_indices, val_indices)
# split_indices = np.append(train_indices, test_indices)
print(split_indices)
ps = PredefinedSplit(test_fold=split_indices)

[-1 -1 -1 ...  0  0  0]


In [10]:
for i, (train_index, test_index) in enumerate(ps.split()):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

Fold 0:
  Train: index=[    0     1     2 ... 18560 18561 18562]
  Test:  index=[18563 18564 18565 ... 20980 20981 20982]


In [9]:
# xgboost_tuning(train, val)

In [15]:
y_train_val = train_val['y_pred']
X_train_val = train_val.drop(['y_pred'], axis=1)
y_train = train['y_pred']
X_train = train.drop(['y_pred'], axis=1)
y_val = val['y_pred']
X_val = val.drop(['y_pred'], axis=1)
# Define the hyperparameters to tune
param_distributions = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5, 6, 7],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'reg_alpha': [0.0, 0.1, 0.2],
    'reg_lambda': [0.0, 0.1, 0.2]
}
# Create a RandomizedSearchCV object
cv = RandomizedSearchCV(xgb.XGBClassifier(), param_distributions, scoring='accuracy', n_iter=10, cv=ps, n_jobs=-1, refit=False)
# Fit the model to the training data
cv.fit(X_train_val, y_train_val)

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type...
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.8, 0.9, 1.0],
                                        'gamma': [0, 0.1, 0.2],
                                        'learning_rate': [0.01, 0.05, 0.1],
                                        'max_depth': [3, 4, 5, 6, 7],
                                        'n_estimators': [100, 200, 300],
                                        'reg_alpha': [0.0, 0.1, 0.2],
                                        'reg_lambda': [0.0, 0.1, 0.2],
                                        'subsample': [0.5, 0.7, 1.0]},
                   refit=False, scoring='accuracy')

In [16]:
best_params = cv.best_params_

In [18]:
xg_clf = xgb.XGBClassifier(objective='binary:logistic', **best_params)
xg_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [19]:
cv.cv_results_

{'mean_fit_time': array([ 7.20125008, 10.72580504,  5.21954322,  6.49166918,  3.39758492,
         9.12353921,  3.77387595,  7.94310808,  9.21854472, 13.93523836]),
 'std_fit_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'mean_score_time': array([0.00502682, 0.00665283, 0.01003885, 0.00813174, 0.00282788,
        0.00636983, 0.002949  , 0.00630522, 0.00582123, 0.0110929 ]),
 'std_score_time': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'param_subsample': masked_array(data=[0.5, 1.0, 0.5, 1.0, 1.0, 0.5, 1.0, 0.5, 0.7, 0.5],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_reg_lambda': masked_array(data=[0.2, 0.2, 0.0, 0.2, 0.1, 0.2, 0.1, 0.1, 0.1, 0.1],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_reg_alpha': masked_array(data

In [20]:
def get_top_n_hyperparameters(cv_results, n):
    results = pd.DataFrame(cv_results)
    results = results.sort_values(by='rank_test_score')
    return results[:n]

In [21]:
def get_dict_top_n_hyperparameters(cv_results, n):
    ''' 
    returns a dict of top n hyperparameters
    '''
    results = pd.DataFrame(cv_results)
    results = results.sort_values(by='rank_test_score')
    results = results[:n]
    return results.to_dict('records')


In [23]:
hyperparameters_dict = get_dict_top_n_hyperparameters(cv.cv_results_, 3)
hyperparameters_dict

[{'mean_fit_time': 7.201250076293945,
  'std_fit_time': 0.0,
  'mean_score_time': 0.005026817321777344,
  'std_score_time': 0.0,
  'param_subsample': 0.5,
  'param_reg_lambda': 0.2,
  'param_reg_alpha': 0.1,
  'param_n_estimators': 200,
  'param_max_depth': 4,
  'param_learning_rate': 0.1,
  'param_gamma': 0,
  'param_colsample_bytree': 0.9,
  'params': {'subsample': 0.5,
   'reg_lambda': 0.2,
   'reg_alpha': 0.1,
   'n_estimators': 200,
   'max_depth': 4,
   'learning_rate': 0.1,
   'gamma': 0,
   'colsample_bytree': 0.9},
  'split0_test_score': 0.5347107438016528,
  'mean_test_score': 0.5347107438016528,
  'std_test_score': 0.0,
  'rank_test_score': 1},
 {'mean_fit_time': 3.7738759517669678,
  'std_fit_time': 0.0,
  'mean_score_time': 0.0029489994049072266,
  'std_score_time': 0.0,
  'param_subsample': 1.0,
  'param_reg_lambda': 0.1,
  'param_reg_alpha': 0.1,
  'param_n_estimators': 100,
  'param_max_depth': 3,
  'param_learning_rate': 0.05,
  'param_gamma': 0,
  'param_colsample_byt

In [53]:
get_top_n_hyperparameters(cv.cv_results_, 5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_reg_lambda,param_reg_alpha,param_n_estimators,param_max_depth,param_learning_rate,param_gamma,param_colsample_bytree,params,split0_test_score,mean_test_score,std_test_score,rank_test_score
1,9.950652,0.0,0.005183,0.0,1.0,0.0,0.2,300,3,0.01,0.2,0.9,"{'subsample': 1.0, 'reg_lambda': 0.0, 'reg_alp...",0.548347,0.548347,0.0,1
0,20.327328,0.0,0.013661,0.0,0.5,0.0,0.0,300,7,0.05,0.1,1.0,"{'subsample': 0.5, 'reg_lambda': 0.0, 'reg_alp...",0.539669,0.539669,0.0,2
9,10.763351,0.0,0.007768,0.0,0.5,0.0,0.0,200,6,0.05,0.2,1.0,"{'subsample': 0.5, 'reg_lambda': 0.0, 'reg_alp...",0.533058,0.533058,0.0,3
2,11.927452,0.0,0.006695,0.0,0.7,0.0,0.2,300,4,0.05,0,0.9,"{'subsample': 0.7, 'reg_lambda': 0.0, 'reg_alp...",0.528512,0.528512,0.0,4
5,10.648801,0.0,0.006432,0.0,0.5,0.0,0.1,200,5,0.05,0.2,1.0,"{'subsample': 0.5, 'reg_lambda': 0.0, 'reg_alp...",0.521488,0.521488,0.0,5


In [45]:
results = cv.cv_results_
for i in range(3):
    print(f"Top {i+1} Hyperparameters:")
    for param, value in best_params.items():
        print(f"{param}: {value}")
    print(f"Mean Validation Score: {results['mean_test_score'][cv.best_index_]:.4f}")
    print("=" * 30)

Top 1 Hyperparameters:
subsample: 1.0
reg_lambda: 0.0
reg_alpha: 0.2
n_estimators: 300
max_depth: 3
learning_rate: 0.01
gamma: 0.2
colsample_bytree: 0.9
Mean Validation Score: 0.5483
Top 2 Hyperparameters:
subsample: 1.0
reg_lambda: 0.0
reg_alpha: 0.2
n_estimators: 300
max_depth: 3
learning_rate: 0.01
gamma: 0.2
colsample_bytree: 0.9
Mean Validation Score: 0.5483
Top 3 Hyperparameters:
subsample: 1.0
reg_lambda: 0.0
reg_alpha: 0.2
n_estimators: 300
max_depth: 3
learning_rate: 0.01
gamma: 0.2
colsample_bytree: 0.9
Mean Validation Score: 0.5483


In [27]:
def xgboost_tuning_random(train_val, n_experiments=10, ps=ps, top_n=3):
    '''
    
    returns a dict of top n hyperparameters
    n_experiments: number of random experiments
    ps: PredefinedSplit object
    top_n: number of top results to return

    '''

    y_train_val = train_val['y_pred']
    X_train_val = train_val.drop(['y_pred'], axis=1)

    # Define the hyperparameters to tune
    param_distributions = {
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5, 6, 7],
        'gamma': [0, 0.1, 0.2],
        'subsample': [0.5, 0.7, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0],
        'reg_alpha': [0.0, 0.1, 0.2],
        'reg_lambda': [0.0, 0.1, 0.2]
    }

    # Create a RandomizedSearchCV object
    cv = RandomizedSearchCV(xgb.XGBClassifier(), param_distributions, scoring='accuracy', n_iter=n_experiments, cv=ps, n_jobs=-1, refit=False)

    # Fit the model to the training data
    cv.fit(X_train_val, y_train_val)

    results = pd.DataFrame(cv.cv_results_)
    results = results.sort_values(by='rank_test_score')
    results = results[:top_n]
    return results.to_dict('records')

In [28]:
top_models = xgboost_tuning_random(train_val, n_experiments=10)

In [31]:
print(top_models[0]['mean_test_score'])
xgb_model = xgb.XGBClassifier(objective='binary:logistic', **top_models[0])
xgb_model.fit(train_val.drop(['y_pred'], axis=1), train_val['y_pred'])

0.5376033057851239
[12:47:38] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "mean_fit_time", "mean_score_time", "mean_test_score", "param_colsample_bytree", "param_gamma", "param_learning_rate", "param_max_depth", "param_n_estimators", "param_reg_alpha", "param_reg_lambda", "param_subsample", "params", "rank_test_score", "split0_test_score", "std_fit_time", "std_score_time", "std_test_score" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              mean_fit_time=8.921621799468994,
              mean_score_time=0.006334066390991211,
              mean_test_score=0.5376033057851239, min_child_weight=None,
              missing=nan, monotone_constraints=None, n_estimators=100,
              n_jobs=None, ...)

In [32]:
y_pred = xgb_model.predict_proba(test.drop(['y_pred'], axis=1))

In [35]:
y_pred[:,1]

array([0.7420748 , 0.7828538 , 0.19410396, ..., 0.49151057, 0.47047368,
       0.4656662 ], dtype=float32)